In [ ]:
#Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.
#SPDX-License-Identifier: MIT-0

In [ ]:
!pip install -q boto3==1.34.149

In [ ]:
import boto3
import importlib

#adding our utils library to sys path
import sys
sys.path.append("../src/utils/")
import llm_utils

importlib.reload(llm_utils)

bedrock_agent_client = boto3.client('bedrock-agent')

### Retrieve the transcript for testing later

In [ ]:
transcripts = llm_utils.load_jsonlines_file("../generated/transcripts/transcripts.jsonl")
transcripts[0]

# Amazon Bedrock Prompt Management
At the time this workshop is created, Amazon Bedrock Prompt Management is a recently released feature still in preview. At the moment, it does not support system_prompt and would require the creation of 2 prompt templates. 
Considering this, we're storing the whole prompt including the system prompt within one single prompt template.

Also note that we're only creating the prompt in this notebook. We will "deploy" the prompt by creating a version in notebook 4.

# Prompts that will be used for optimisation and evaluation

## Sentiment categorisation prompt

In [ ]:
sentiment_prompt_template = """
You are a professional detailed oriented sentiment analyser.

Your task is to categorise the sentiment from the following conversation transcript between a customer and a call centre agent in <transcript> tag.

<transcript>
{transcript}
</transcript>

Think of the problem step by step:
1. Read carefully through the transcript.
2. Review the examples in <examples> tag to calibrate your response.
3. Categorise the conversation's sentiment into one of the following categories:
    - Very Positive
    - Positive
    - Neutral
    - Negative
    - Very Negative
4. Skip the preamble and provide your response in <answer> tag.

<examples>
    <example>
        <example_transcript>Hi, I'm calling to let you know that your service is exceptional, I love the variety and the overall UI. I would have just a recommendation to add a favourite functionality to your features.</example_transcript>
        <answer>Very Positive<answer>
    </example>
    <example>
        <example_transcript>Hi, I'm calling to inquire about any promotional offers or discounts you might have available.</example_transcript>
        <answer>Positive<answer>
    </example>
    <example>
        <example_transcript>Hi, I need to cancel my account. I'm moving to a different city, and your service isn't available there.</example_transcript>
        <answer>Neutral<answer>
    </example>
    <example>
        <example_transcript>I've been a loyal customer for years, and I'm getting really impatient with the lack of new content.</example_transcript>
        <answer>Negative<answer>
    </example>
    <example>
        <example_transcript>I'm so angry right now! The video quality on your streaming service is terrible. It keeps buffering and pixelating, making it impossible to watch anything.</example_transcript>
        <answer>Very Negative<answer>
    </example>
<examples>
"""

We use Amazon Bedrock prompt management to store the original  version of our prompt

In [ ]:
sentiment_categorisation_model_id = "anthropic.claude-3-sonnet-20240229-v1:0"
sentiment_prompt_name = "sentiment_categorisation"
try:
    sentiment_prompt_response = bedrock_agent_client.create_prompt(
        defaultVariant='v1',
        description='prompt used in the context of sentiment categorisation of a conversation between a user and a call centre agent',
        name=sentiment_prompt_name,
        tags={
            'project-name': 'evaluation_workshop'
        },
        variants=[
            {
                'inferenceConfiguration': {
                    'text': {
                        'maxTokens': 400,
                        'stopSequences': [
                            '</answer>',
                        ],
                        'temperature': 0,
                        'topP': 0.8
                    }
                },
                'modelId': sentiment_categorisation_model_id,
                'name': 'v1',
                'templateConfiguration': {
                    'text': {
                        'inputVariables': [
                            {
                                'name': 'transcript'
                            },
                        ],
                        'text': sentiment_prompt_template
                    }
                },
                'templateType': 'TEXT'
            },
        ]
    )

except Exception as e:
    print("Prompt with similar name already exists, retrieving prompt instead")

    #get id for the already created prompt
    id =""
    list_prompts = bedrock_agent_client.list_prompts(maxResults=10)
    for prompt in list_prompts['promptSummaries']:
        if prompt["name"] == sentiment_prompt_name:
            id = prompt["id"]
            break
    
    #retrieve prompt
    sentiment_prompt_response = bedrock_agent_client.get_prompt(
        promptIdentifier=id
    )

sentiment_categorisation_prompt_id = sentiment_prompt_response['id']
sentiment_categorisation_prompt_arn = sentiment_prompt_response['arn']

print(f"Sentiment prompt id: {sentiment_categorisation_prompt_id}")
print(f"Sentiment prompt arn: {sentiment_categorisation_prompt_arn}")

## Summarisation prompt

In [ ]:
summarisation_prompt_template = """
You are a highly skilled and professional writer.

Your task is to summarise the conversation between a customer and a call centre agent in <transcript> tag.

<transcript>
{transcript}
</transcript>

Read carefully through the transcript.

Make sure to mention the important parts of the conversation including: reason for the call, problems encountered by the customer, tone of the conversation, resolution and next steps.

Provide the summary in a <answer> tag.
"""

We use Amazon Bedrock prompt management to store the original  version of our prompt

In [ ]:
summarisation_model_id = "anthropic.claude-3-sonnet-20240229-v1:0"
summarisation_prompt_name = "summarisation"
try:
    summarisation_prompt_response = bedrock_agent_client.create_prompt(
        defaultVariant='v1',
        description='prompt used in the context of summarisation of a conversation between a user and a call centre agent',
        name=summarisation_prompt_name,
        tags={
            'project-name': 'evaluation_workshop'
        },
        variants=[
            {
                'inferenceConfiguration': {
                    'text': {
                        'maxTokens': 2048,
                        'stopSequences': [
                            '</answer>',
                        ],
                        'temperature': 0,
                        'topP': 0.8
                    }
                },
                'modelId': summarisation_model_id,
                'name': 'v1',
                'templateConfiguration': {
                    'text': {
                        'inputVariables': [
                            {
                                'name': 'transcript'
                            },
                        ],
                        'text': summarisation_prompt_template
                    }
                },
                'templateType': 'TEXT'
            },
        ]
    )
except Exception as e:
    print("Prompt with similar name already exists, retrieving prompt instead")

    #get id for the already created prompt
    id =""
    list_prompts = bedrock_agent_client.list_prompts(maxResults=10)
    for prompt in list_prompts['promptSummaries']:
        if prompt["name"] == summarisation_prompt_name:
            id = prompt["id"]
            break
    
    #retrieve prompt
    summarisation_prompt_response = bedrock_agent_client.get_prompt(
        promptIdentifier=id
    )

summarisation_prompt_id = summarisation_prompt_response['id']
summarisation_prompt_arn = summarisation_prompt_response['arn']

print(f"Summarisation prompt id: {summarisation_prompt_id}")
print(f"Summarisation prompt arn: {summarisation_prompt_arn}")

## Theme extraction prompt

In [ ]:
extraction_prompt_template = """
You are an intelligent information extraction assistant.

Your task is to analyse the transcript of a conversation between a customer and a call centre agent in <transcript> tag and extract the key themes.

<transcript>
{transcript}
</transcript>

Read carefully through the transcript.

Provide <number>3</number> key themes or topics that are important in the conversation in <answer> tag as shown in the example below.

<example>
    <answer>technical issue, lack of transparence of policy, business impact</answer>
</example>
"""

We use Amazon Bedrock prompt management to store the original  version of our prompt

In [ ]:
extraction_model_id = "anthropic.claude-3-sonnet-20240229-v1:0"

extraction_prompt_name = "theme_extraction"

try:
    extraction_prompt_response = bedrock_agent_client.create_prompt(
        defaultVariant='v1',
        description='prompt used in the context of theme extraction from a conversation between a user and a call centre agent',
        name=extraction_prompt_name,
        tags={
            'project-name': 'evaluation_workshop'
        },
        variants=[
            {
                'inferenceConfiguration': {
                    'text': {
                        'maxTokens': 2048,
                        'stopSequences': [
                            '</answer>',
                        ],
                        'temperature': 0,
                        'topP': 0.8
                    }
                },
                'modelId': extraction_model_id,
                'name': 'v1',
                'templateConfiguration': {
                    'text': {
                        'inputVariables': [
                            {
                                'name': 'transcript'
                            },
                        ],
                        'text': extraction_prompt_template
                    }
                },
                'templateType': 'TEXT'
            },
        ]
    )
except Exception as e:
    print("Prompt with similar name already exists, retrieving prompt instead")

    #get id for the already created prompt
    id =""
    list_prompts = bedrock_agent_client.list_prompts(maxResults=10)
    for prompt in list_prompts['promptSummaries']:
        if prompt["name"] == extraction_prompt_name:
            id = prompt["id"]
            break
    
    #retrieve prompt
    extraction_prompt_response = bedrock_agent_client.get_prompt(
        promptIdentifier=id
    )

extraction_prompt_id = extraction_prompt_response['id']
extraction_prompt_arn = extraction_prompt_response['arn']

print(f"Extraction prompt id: {extraction_prompt_id}")
print(f"Extraction prompt arn: {extraction_prompt_arn}")

List all created prompts

In [ ]:
response = bedrock_agent_client.list_prompts(
    maxResults=10
)
response['promptSummaries']


In [ ]:
%store sentiment_categorisation_prompt_id
%store sentiment_categorisation_prompt_arn
%store summarisation_prompt_id
%store summarisation_prompt_arn
%store extraction_prompt_id
%store extraction_prompt_arn
